In [2]:
from collections import defaultdict, Counter
import pandas as pd
match = pd.read_csv('all_matches.csv')
stat = pd.read_csv('all_stats.csv')
tourney = pd.read_csv('all_tournaments.csv')
time = pd.read_csv('score_by_minute.csv')
print(time.head())

         dataset  year  match_id    team  minute_score
0  1930--uruguay  1930         1  France            19
1  1930--uruguay  1930         1  France            40
2  1930--uruguay  1930         1  France            43
3  1930--uruguay  1930         1  France            87
4  1930--uruguay  1930         1  Mexico            70


In [3]:
# peek at the data
print(match.head())
print('-'*20)
print(stat.head())
print('-'*20)
print(tourney.head())
print('-'*20)
print(time.head())

         dataset  year  match_id  home_team away_team  home_score  away_score  \
0  1930--uruguay  1930         1     France    Mexico           4           4   
1  1930--uruguay  1930         5  Argentina    France           1           1   
2  1930--uruguay  1930         6      Chile    Mexico           3           3   
3  1930--uruguay  1930        10      Chile    France           1           1   
4  1930--uruguay  1930        11  Argentina    Mexico           6           6   

      winner  
0     France  
1  Argentina  
2      Chile  
3      Chile  
4  Argentina  
--------------------
     team  year  win  draw  loss  goal_for  goal_against
0  France  1930    1     0     2         4             3
1  France  1934    0     0     1         2             3
2  France  1938    1     0     1         4             4
3  France  1954    1     0     1         3             3
4  France  1958    4     0     2        23            15
--------------------
             dataset  year     champion

In [5]:
match.head()

,dataset,year,match_id,home_team,away_team,home_score,away_score,winner
0,1930--uruguay,1930,1,France,Mexico,4,4,France
1,1930--uruguay,1930,5,Argentina,France,1,1,Argentina
2,1930--uruguay,1930,6,Chile,Mexico,3,3,Chile
3,1930--uruguay,1930,10,Chile,France,1,1,Chile
4,1930--uruguay,1930,11,Argentina,Mexico,6,6,Argentina


In [6]:
# Q: Who wins most matches ?(using data from all_stats)
avgWin = {}
win = {}
for team in stat['team'].unique():
    win[team] = sum(stat[(stat['team'] == team) & (stat['year'] > 0)]['win'])
    avgWin[team] = win[team]/stat[stat['team'] == team].shape[0]
print('Most wins:',Counter(win).most_common(5))
print('-'*40
print('Most Average per year wins:',Counter(avgWin).most_common(5))

Most wins: [('Brazil', 76), ('Argentina', 47), ('Italy', 46), ('West Germany', 39), ('France', 36)]
Most Average per year wins: [('West Germany', 3.9), ('Brazil', 3.619047619047619), ('Germany', 3.5555555555555554), ('Ukraine', 3.0), ('Netherlands', 2.8)]


In [7]:
# Who scores most goal ? (using data from all_stats)
avgGoal = {}
goals = {}
for team in stat['team'].unique():
    allGoals = sum(stat[stat['team'] == team]['goal_for'])
    numYear = stat[stat['team'] == team].shape[0]
    goals[team] = allGoals
    avgGoal[team] = allGoals/numYear
avgGoal = Counter(avgGoal)
goals = Counter(goals)
print('Top Average Goal:', avgGoal.most_common(10))
print('-'*40)
print('Top total goals:', goals.most_common(10))

Top Average Goal: [('West Germany', 13.1), ('Brazil', 10.904761904761905), ('Germany', 10.555555555555555), ('Turkey', 10.0), ('Hungary', 9.666666666666666), ('Netherlands', 8.6), ('Argentina', 8.058823529411764), ('France', 8.0), ('Soviet Union', 7.571428571428571), ('Italy', 7.111111111111111)]
----------------------------------------
Top total goals: [('Brazil', 229), ('Argentina', 137), ('West Germany', 131), ('Italy', 128), ('France', 120), ('Spain', 99), ('Germany', 95), ('England', 91), ('Uruguay', 87), ('Hungary', 87)]


In [ ]:
# Who score the most goal? (Using data from all_matches)
from collections import defaultdict, Counter
NYear = defaultdict(int)
NGoal = defaultdict(int)
AvgGoal = defaultdict(float)
check = defaultdict(defaultdict)
for idx, row in match.iterrows():
    for x in ['home', 'away']:
        if (row['dataset'] not in check):
            check[row['dataset']] = defaultdict(int)
        if (check[row['dataset']][row['%s_team'%x]] == 0):
            check[row['dataset']][row['%s_team'%x]] = 1
            NYear[row['%s_team'%x]] += 1
        NGoal[row['%s_team'%x]] += row['%s_score'%x]
for team in NYear:
    AvgGoal[team] = NGoal[team]/NYear[team]
print(Counter(AvgGoal).most_common(10))

In [20]:
# Q: Which teams performs better in second half?(using data from score_by_minute)
second_half = time[(time['minute_score'] > 45) & (time['minute_score'] <= 90)]['team'].value_counts().to_dict()
first_half = time[time['minute_score'] <= 45]['team'].value_counts().to_dict()
ratio = {}
diff = {}
for team in second_half:
    # if not filter out small top data 
    if (team in first_half and second_half[team] + first_half[team] >= 10):
        diff[team] = second_half[team]-first_half[team]
        ratio[team] = (second_half[team]/first_half[team], second_half[team], first_half[team])
print(Counter(ratio).most_common(10))
print('-'*20)
print(Counter(diff).most_common(10))

[('Mexico', (3.6666666666666665, 11, 3)), ('Czechoslovakia', (2.6666666666666665, 8, 3)), ('Colombia', (2.6, 13, 5)), ('Netherlands', (2.4, 12, 5)), ('Croatia', (2.0, 12, 6)), ('Yugoslavia', (1.8, 9, 5)), ('Belgium', (1.5555555555555556, 14, 9)), ('Brazil', (1.5217391304347827, 35, 23)), ('Russia', (1.4, 7, 5)), ('Switzerland', (1.3636363636363635, 15, 11))]
--------------------
[('Brazil', 12), ('Colombia', 8), ('Mexico', 8), ('Netherlands', 7), ('Croatia', 6), ('Belgium', 5), ('Czechoslovakia', 5), ('Switzerland', 4), ('Yugoslavia', 4), ('United States', 2)]


In [16]:
#Q: which team win in highest number of continents?
print(tourney['champion'].value_counts())

Italy               4
Brazil              4
Argentina           2
Uruguay             2
France              2
West Germany        2
Germany             1
Switzerland         1
Spain               1
England             1
Northern Ireland    1
Name: champion, dtype: int64


In [11]:
tourney[tourney['champion'] == 'Brazil']

,dataset,year,champion,runner-up
6,1962--chile,1962,Brazil,Czechoslovakia
8,1970--mexico,1970,Brazil,Italy
14,1994--united-states,1994,Brazil,Italy
16,2002--south-korea-n-japan,2002,Brazil,Germany


In [8]:
#Bonus Q: Does the first-5-minute and last-5-minute get "a lot" of goals 
print(time[time['minute_score'] <= 5].count())
print(time[(40 <= time['minute_score']) & (time['minute_score'] <= 45)].count())
print(time[time['minute_score'] <= 45].count())

dataset         25
year            25
match_id        25
team            25
minute_score    25
dtype: int64
dataset         40
year            40
match_id        40
team            40
minute_score    40
dtype: int64
dataset         276
year            276
match_id        276
team            276
minute_score    276
dtype: int64


In [14]:
#Q: Does the first-5-minute and last-5-minute get "a lot" of goals (In second half)
print(time[(45 <= time['minute_score']) & (time['minute_score'] <= 45)].count())
print(time[(85 <= time['minute_score']) & (time['minute_score'] <= 90)].count())
print(time[(45 <= time['minute_score']) & (time['minute_score'] <= 90)].count())

dataset         9
year            9
match_id        9
team            9
minute_score    9
dtype: int64
dataset         74
year            74
match_id        74
team            74
minute_score    74
dtype: int64
dataset         360
year            360
match_id        360
team            360
minute_score    360
dtype: int64
